In [ ]:
#paso opcional: este codigo no es necesario ejecutar, ya que el entrenamiento del modelo ya esta guardado, 
# pero si se desea agregar mas informacion a la data para el entrenamiento, se puede volver a entrenar
import torch
from sklearn.metrics import classification_report
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import pandas as pd

MODEL_NAME = "dccuchile/bert-base-spanish-wwm-uncased"
OUTPUT_DIR = "./modelo_entrenado"

def cargar_datos_entrenamiento():
    df = pd.read_csv('comentarios_entrenamiento.csv', sep=';')
    df.columns = df.columns.str.strip().str.lower()
    df['sentimiento'] = df['sentimiento'].str.lower().str.strip()
    df = df[df['sentimiento'].isin(['positivo', 'negativo'])]
    df['label'] = df['sentimiento'].map({'positivo': 1, 'negativo': 0})
    return Dataset.from_pandas(df[['comentario', 'label']])

def tokenize(batch):
    return tokenizer(batch["comentario"], padding=True, truncation=True)

if __name__ == "__main__":
    tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)
    dataset = cargar_datos_entrenamiento()
    dataset = dataset.train_test_split(test_size=0.2)
    dataset = dataset.map(tokenize, batched=True)

    model = BertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)

    training_args = TrainingArguments(
        output_dir="./results",
        evaluation_strategy="epoch",
        logging_strategy="epoch",
        save_strategy="epoch",
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        num_train_epochs=3,
        weight_decay=0.01,
        logging_dir="./logs",
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset['train'],
        eval_dataset=dataset['test'],
        tokenizer=tokenizer,
    )

    trainer.train()

    print("📊 Evaluando modelo...")
    preds = trainer.predict(dataset['test'])
    y_pred = preds.predictions.argmax(axis=1)
    y_true = preds.label_ids
    print(classification_report(y_true, y_pred, target_names=["negativo", "positivo"]))

    # Guardar modelo entrenado
    model.save_pretrained(OUTPUT_DIR)
    tokenizer.save_pretrained(OUTPUT_DIR)
    print(f"✅ Modelo y tokenizer guardados en {OUTPUT_DIR}")


c:\Users\Administrador\anaconda3\envs\bert-es\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Administrador\anaconda3\envs\bert-es\lib\site-packages\huggingface_hub\file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Map: 100%|██████████| 268/268 [00:00<00:00, 4574.56 examples/s]
c:\Users\Administrador\anaconda3\envs\bert-es\lib\site-packages\huggingface_hub\file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertForSequenceClassification were no

{'loss': 0.2157, 'grad_norm': 0.36178359389305115, 'learning_rate': 3.3333333333333335e-05, 'epoch': 1.0}


                                                 
 33%|███▎      | 134/402 [04:24<09:08,  2.05s/it]

{'eval_loss': 0.06909680366516113, 'eval_runtime': 8.0602, 'eval_samples_per_second': 33.25, 'eval_steps_per_second': 4.218, 'epoch': 1.0}


 67%|██████▋   | 268/402 [08:40<04:08,  1.85s/it]

...

{'eval_loss': 0.08250705152750015, 'eval_runtime': 7.7553, 'eval_samples_per_second': 34.557, 'eval_steps_per_second': 4.384, 'epoch': 3.0}


100%|██████████| 402/402 [13:18<00:00,  1.99s/it]


{'train_runtime': 798.1202, 'train_samples_per_second': 4.029, 'train_steps_per_second': 0.504, 'train_loss': 0.10635785321098062, 'epoch': 3.0}
📊 Evaluando modelo...


100%|██████████| 34/34 [00:07<00:00,  4.74it/s]


              precision    recall  f1-score   support

    negativo       0.99      0.98      0.98       123
    positivo       0.98      0.99      0.99       145

    accuracy                           0.99       268
   macro avg       0.99      0.98      0.98       268
weighted avg       0.99      0.99      0.99       268

✅ Modelo y tokenizer guardados en ./modelo_entrenado


In [ ]:
#´Paso 2: Generacion de clasificacion de profesores
import torch
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification

MODEL_DIR = "./modelo_entrenado"

def clasificar_profesores(model, tokenizer):
    df = pd.read_csv("comentarios/datacoment.csv", sep=';')
    df.columns = df.columns.str.strip()
    df = df.dropna(subset=['comentarios'])

    textos = df['comentarios'].tolist()
    tokens = tokenizer(textos, padding=True, truncation=True, return_tensors="pt")

    with torch.no_grad():
        outputs = model(**tokens)
        preds = torch.argmax(outputs.logits, dim=1).numpy()

    df['sentimiento'] = ['positivo' if p == 1 else 'negativo' for p in preds]

    clasificacion = []
    for profesor, grupo in df.groupby('Docente'):
        positivos = (grupo['sentimiento'] == 'positivo').sum()
        negativos = (grupo['sentimiento'] == 'negativo').sum()

        if positivos > negativos:
            clasif = 'bueno'
        elif negativos > positivos:
            clasif = 'malo'
        else:
            clasif = 'neutro'

        clasificacion.append({'Docente': profesor, 'clasificacion': clasif})

    resultado_df = pd.DataFrame(clasificacion)
    resultado_df.to_csv("clasificacion_profesores.csv", index=False, sep=';')
    print("✅ Clasificación generada en 'clasificacion_profesores.csv'")

if __name__ == "__main__":
    tokenizer = BertTokenizer.from_pretrained(MODEL_DIR)
    model = BertForSequenceClassification.from_pretrained(MODEL_DIR)
    clasificar_profesores(model, tokenizer)


✅ Clasificación generada en 'clasificacion_profesores.csv'


In [ ]:
#Paso 3: ejecutar esto cuando se haya extraido todos los comentarios de los profesores, así se fusionaran con la data de horarios_validos.csv

import pandas as pd
import unicodedata

# Función para normalizar nombres (minúsculas, sin tildes, sin espacios extras)
def normalizar(nombre):
    nombre = unicodedata.normalize('NFKD', nombre).encode('ASCII', 'ignore').decode('utf-8')
    return nombre.lower().strip()

# Leer archivos
horarios = pd.read_csv("horarios_validos.csv", encoding="utf-8")
clasificacion = pd.read_csv("clasificacion_profesores.csv", sep=";", encoding="utf-8")

# Normalizar nombres en ambos DataFrames
horarios["docente_normalizado"] = horarios["Docente"].apply(normalizar)
clasificacion["docente_normalizado"] = clasificacion["Docente"].apply(normalizar)

# Hacer el merge por la columna normalizada
df_final = horarios.merge(
    clasificacion[["docente_normalizado", "clasificacion"]],
    on="docente_normalizado",
    how="left"
)

# Eliminar columna auxiliar y guardar el resultado
df_final.drop(columns=["docente_normalizado"], inplace=True)
df_final.to_csv("horarios_con_clasificacion.csv", index=False, encoding="utf-8")

print("✅ Archivo 'horarios_con_clasificacion.csv' generado con la columna 'clasificacion'.")
